In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc
import imageio
import os
import pandas as pd
import mahotas as mh

In [2]:
# folder path
dir_path = r'BestSegmentations/'
# list to store files
res = [] # files name
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [11]:
dmv_iou_otsu = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'], index=range(370))
for i in range(0, 370, 1):
  segm1 = imageio.imread(('BestSegmentations/' + res[i]))
  res1 = str(res[i])
  dmv_otsu = cv2.imread(('GrayHeatmapsDmvcnn/' + res1.split(".")[0] + '_CAM3.png'), cv2.IMREAD_UNCHANGED)
  assert np.shape(segm1) == np.shape(dmv_otsu)
  otsu_threshold, image_otsu = cv2.threshold(dmv_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
  intersection = np.logical_and(image_otsu, segm1)
  union = np.logical_or(image_otsu, segm1)
  iou = np.sum(intersection) / np.sum(union)
  dmv_iou_otsu.loc[i] = (res1.split(".")[0], otsu_threshold, iou)
  # Save image with contours--------------------------------------------------------------------------------------------
  #orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
  #contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #contours_otsu, hierarchy1 = cv2.findContours(image_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
  #cv2.drawContours(orig, contours_otsu, -1, (255,255,0), 6)
  #cv2.imwrite(('ImageOtsu/Dmvcnn/' + res1.split(".")[0] + '_CAM3OT.png'), orig)

In [13]:
dmv_iou_otsu

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,29,0.145201
1,173ca_256_R_MLO,40,0.0147354
2,168ca_248_L_MLO,35,0.00750082
3,137ca_208_R_CC,48,0.000972933
4,133ca_203_L_MLO,53,0.0107
...,...,...,...
365,14ca_19_R_MLO,38,0.0359078
366,140ca_211_L_MLO,29,0.0934856
367,23ca_38_L_CC,32,0.0186259
368,13ca_18_R_CC,54,0


In [14]:
np.sum(dmv_iou_otsu['IoU'])/370

0.03238392101528104

In [15]:
dmv_iou = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
for i in range(0, 370, 1):
  segm1 = imageio.imread(('BestSegmentations/' + res[i]))
  res1 = str(res[i])
  dmv = cv2.imread(('GrayHeatmapsDmvcnn/' + res1.split(".")[0] + '_CAM3.png'), cv2.IMREAD_UNCHANGED)
  assert np.shape(segm1) == np.shape(dmv)
  iou_score = []
  for n in range(0, 255, 1):
    th , image = cv2.threshold(dmv, n, 255, cv2.THRESH_BINARY) 
    intersection = np.logical_and(image, segm1)
    union = np.logical_or(image, segm1)
    iou = np.sum(intersection) / np.sum(union)
    iou_score.append(iou)
  threshold = iou_score.index(np.max(iou_score))
  IoU = np.max(iou_score)
  dmv_iou.loc[i] = (res1.split(".")[0], threshold, IoU)
  # Save image with contours--------------------------------------------------------------------------------------------
  orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
  th , image = cv2.threshold(dmv, threshold, 255, cv2.THRESH_BINARY)
  contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours_sweep, hierarchy1 = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
  cv2.drawContours(orig, contours_sweep, -1, (255,255,0), 6)
  cv2.imwrite(('ImageSweep/Gmic/' + res1.split(".")[0] + '_CAM3SW.png'), orig)

In [ ]:
dmv_iou

In [ ]:
np.sum(dmv_iou['IoU'])/370

0.20541965270922985

In [3]:
dmv_iou_otsu_ = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'])
dmv_iou_no_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
otsu_ = 0
no_ = 0
for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    dmv_otsu = cv2.imread(('GrayHeatmapsDmvcnn/' + res1.split(".")[0] + '_CAM3.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(dmv_otsu)
    otsu_threshold, image_otsu = cv2.threshold(dmv_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    inter_out = np.sum(np.logical_and(image_otsu, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_otsu)
        new_image = np.zeros(np.shape(image_otsu))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        dmv_iou_otsu_.loc[otsu_] = (res1.split(".")[0], otsu_threshold, iou)
        otsu_ += 1
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageOtsu/Dmvcnn/' + res1.split(".")[0] + '_CAM3OT.png'), orig)
    else:
        dmv_iou_no_.loc[no_] = (res1.split(".")[0], otsu_threshold)
        no_ += 1

In [4]:
dmv_iou_otsu_

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,29.0,0.179569
1,173ca_256_R_MLO,40.0,0.014735
2,168ca_248_L_MLO,35.0,0.007501
3,137ca_208_R_CC,48.0,0.000973
4,133ca_203_L_MLO,53.0,0.010700
...,...,...,...
292,154ca_228_L_CC,42.0,0.057806
293,14ca_19_R_MLO,38.0,0.036448
294,140ca_211_L_MLO,29.0,0.093486
295,23ca_38_L_CC,32.0,0.018678


In [1]:
print('IoU Dmvcnn Otsu = ', np.sum(dmv_iou_otsu_['IoU'])/otsu_)
print(otsu_)

NameError: name 'np' is not defined

In [6]:
dmv_iou_sw_ = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
dmv_iou_n_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
sw_ = 0
n_ = 0

for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    dmv_sw = cv2.imread(('GrayHeatmapsDmvcnn/' + res1.split(".")[0] + '_CAM3.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(dmv_sw)
    iou_score = []
    for n in range(0, 255, 1):
        th , image = cv2.threshold(dmv_sw, n, 255, cv2.THRESH_BINARY)      
        intersection = np.logical_and(image, segm1)
        union = np.logical_or(image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        iou_score.append(iou)
    threshold = iou_score.index(np.max(iou_score))
    IoU = np.max(iou_score)
    th , image_sw = cv2.threshold(dmv_sw, threshold, 255, cv2.THRESH_BINARY) 
    inter_out = np.sum(np.logical_and(image_sw, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_sw)
        new_image = np.zeros(np.shape(image_sw))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        dmv_iou_sw_.loc[sw_] = (res1.split(".")[0], th, iou)
        sw_ += 1
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageSweep/Dmvcnn/' + res1.split(".")[0] + '_CAM3SW.png'), orig)
    else:
        dmv_iou_n_.loc[n_] = (res1.split(".")[0], otsu_threshold)
        n_ += 1

In [7]:
dmv_iou_sw_

,image_index,threshold,IoU
0,123ca_187_L_CC,75.0,0.478915
1,173ca_256_R_MLO,135.0,0.678175
2,168ca_248_L_MLO,78.0,0.360143
3,137ca_208_R_CC,141.0,0.013900
4,133ca_203_L_MLO,107.0,0.015081
...,...,...,...
292,154ca_228_L_CC,114.0,0.510029
293,14ca_19_R_MLO,75.0,0.625961
294,140ca_211_L_MLO,71.0,0.330627
295,23ca_38_L_CC,98.0,0.427916


In [8]:
print('IoU Dmvcnn Sweep = ', np.sum(dmv_iou_sw_['IoU'])/sw_)

IoU Dmvcnn Sweep =  0.3335187300241108
